In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install qiskit qiskit-aer qiskit-machine-learning qiskit-algorithms qiskit-ibm-provider qiskit-ibm-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.7/368.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/7

## Summary:

### Data Analysis Key Findings

*   The dataset consists of transaction and identity information, merged successfully based on `TransactionID`, resulting in 590,540 rows and 434 columns.
*   A significant portion of columns (those with >50% missing values) were dropped during preprocessing to handle missing data. Remaining missing values were imputed using the median for numerical columns and the mode for categorical columns, resulting in a dataframe with no missing values.
*   Categorical features were successfully encoded using Label Encoding, converting 'object' type columns to numerical types.
*   Numerical features (excluding 'TransactionID' and 'isFraud') were scaled using `MinMaxScaler`.
*   Feature selection was performed using PCA, reducing the dimensionality to 3 principal components to align with potential qubit limitations for quantum processing.
*   The data was split into training (80%) and testing (20%) sets using stratification to preserve the fraud ratio. A smaller stratified subset of the training data (5000 samples) was created for demonstration.
*   Class imbalance in the training subset was addressed using `RandomOverSampler`, increasing the number of samples.
*   A classical Logistic Regression model was trained on the resampled training subset and evaluated on the test set, achieving a ROC-AUC of 0.7250, an accuracy of 0.7180, a precision of 0.0747, and a recall of 0.6196 for the fraud class.
*   Attempts to set up and train a Qiskit Variational Quantum Classifier (VQC) failed repeatedly due to persistent `ImportError` issues with the `COBYLA` optimizer across multiple attempted import paths (`qiskit.algorithms.optimizers`, `qiskit.optimize`, `qiskit.utils.algorithm_globals`).
*   Consequently, the VQC could not be trained or evaluated, making a direct performance comparison between the classical and quantum models impossible within this process.

### Insights or Next Steps

*   The primary bottleneck was the inability to import the required Qiskit optimizer. Resolving this library compatibility issue is the critical next step to enable VQC training and proceed with the hybrid quantum-classical pipeline.
*   Once the VQC training is functional, future steps should include hyperparameter tuning for both the classical and VQC models, exploring different feature selection methods (potentially involving more features if qubit limits allow), and potentially experimenting with different Qiskit feature maps and ansatz circuits to optimize VQC performance.


new code


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Load data
df_identity = pd.read_csv('/content/drive/MyDrive/train_identity.csv')
df_transaction = pd.read_csv('/content/drive/MyDrive/train_transaction.csv')

# Merge data
df_merged = pd.merge(df_transaction, df_identity, on='TransactionID', how='outer')

# Handle missing values
missing_percentage = df_merged.isnull().sum() / len(df_merged) * 100
missing_threshold = 50
cols_to_drop = missing_percentage[missing_percentage > missing_threshold].index
df_merged_cleaned = df_merged.drop(columns=cols_to_drop)

for col in df_merged_cleaned.columns:
    if df_merged_cleaned[col].isnull().any():
        if df_merged_cleaned[col].dtype in ['int64', 'float64']:
            df_merged_cleaned[col] = df_merged_cleaned[col].fillna(df_merged_cleaned[col].median())
        else:
            df_merged_cleaned[col] = df_merged_cleaned[col].fillna(df_merged_cleaned[col].mode()[0])

# Encode categorical features
categorical_cols = df_merged_cleaned.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    df_merged_cleaned[col] = le.fit_transform(df_merged_cleaned[col])

# Normalize numerical features
numerical_cols = df_merged_cleaned.select_dtypes(include=['int64', 'float64']).columns
cols_to_scale = numerical_cols.drop(['TransactionID', 'isFraud'])
scaler = MinMaxScaler()
df_merged_cleaned[cols_to_scale] = scaler.fit_transform(df_merged_cleaned[cols_to_scale])

# Now proceed with the original feature selection steps
# 1. Identify the target variable 'isFraud' and separate it from the features
X = df_merged_cleaned.drop(columns=['TransactionID', 'isFraud'])
y = df_merged_cleaned['isFraud']

# 2. Calculate the correlation matrix of the features with the target variable
# We need to combine X and y temporarily to calculate correlations easily
df_with_target = pd.concat([X, y], axis=1)
correlations = df_with_target.corr()['isFraud'].drop('isFraud') # Calculate correlation and drop the target's self-correlation

# 3. Select the top N features with the highest absolute correlation with 'isFraud'
N = 8 # Choose a small number of features suitable for VQC
top_features = correlations.abs().sort_values(ascending=False).head(N).index.tolist()

print(f"Top {N} features based on absolute correlation with 'isFraud': {top_features}")

# 4. Create a new DataFrame containing only these selected features and the target variable
df_selected_features = df_merged_cleaned[top_features + ['isFraud']]

# 5. Display the head and shape of the new DataFrame with selected features
display(df_selected_features.head())
display(df_selected_features.shape)

Top 8 features based on absolute correlation with 'isFraud': ['V45', 'V86', 'V87', 'V44', 'V52', 'V51', 'V40', 'V79']


,V45,V86,V87,V44,V52,V51,V40,V79,isFraud
0,0.020833,0.033333,0.033333,0.020833,0.0,0.0,0.0,0.0,0
1,0.020833,0.033333,0.033333,0.020833,0.0,0.0,0.0,0.0,0
2,0.020833,0.033333,0.033333,0.020833,0.0,0.0,0.0,0.0,0
3,0.020833,0.033333,0.033333,0.020833,0.0,0.0,0.0,0.0,0
4,0.020833,0.033333,0.033333,0.020833,0.0,0.0,0.0,0.0,0


(590540, 9)

## Refined data preprocessing and feature selection

### Subtask:
Revisit the preprocessing steps. Given the constraints of quantum hardware, we need to be very selective about the features used. This might involve exploring more advanced feature selection methods beyond basic PCA or selecting features based on domain knowledge or other statistical methods to find the most impactful features that can be represented on a limited number of qubits.


**Reasoning**:
Separate the target variable, calculate the correlation of features with the target, select the top N features based on absolute correlation, and create a new DataFrame with these features and the target.



In [ ]:
# 1. Identify the target variable 'isFraud' and separate it from the features
X = df_merged_cleaned.drop(columns=['TransactionID', 'isFraud'])
y = df_merged_cleaned['isFraud']

# 2. Calculate the correlation matrix of the features with the target variable
# We need to combine X and y temporarily to calculate correlations easily
df_with_target = pd.concat([X, y], axis=1)
correlations = df_with_target.corr()['isFraud'].drop('isFraud') # Calculate correlation and drop the target's self-correlation

# 3. Select the top N features with the highest absolute correlation with 'isFraud'
N = 4 # Choose a small number of features suitable for VQC
top_features = correlations.abs().sort_values(ascending=False).head(N).index.tolist()

print(f"Top {N} features based on absolute correlation with 'isFraud': {top_features}")

# 4. Create a new DataFrame containing only these selected features and the target variable
df_selected_features = df_merged_cleaned[top_features + ['isFraud']]

# 5. Display the head and shape of the new DataFrame with selected features
display(df_selected_features.head())
display(df_selected_features.shape)

Top 4 features based on absolute correlation with 'isFraud': ['V45', 'V86', 'V87', 'V44']


,V45,V86,V87,V44,isFraud
0,0.020833,0.033333,0.033333,0.020833,0
1,0.020833,0.033333,0.033333,0.020833,0
2,0.020833,0.033333,0.033333,0.020833,0
3,0.020833,0.033333,0.033333,0.020833,0
4,0.020833,0.033333,0.033333,0.020833,0


(590540, 5)

## Advanced qiskit vqc setup

### Subtask:
Define a more complex and potentially more expressive VQC circuit (feature map and ansatz) that is still feasible within the qubit limits of the target IBM Quantum hardware. This could involve exploring different circuit architectures and entanglement strategies.


**Reasoning**:
Define a more complex feature map and ansatz for the VQC circuit using Qiskit's circuit library, combine them, and then draw the resulting circuit.



**Reasoning**:
The previous command failed because the qiskit library was not found. Reinstall the necessary qiskit packages.



In [ ]:
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2
from qiskit import QuantumCircuit

# Assuming N, the number of features from the previous (failed) step, is defined.
# Since the previous step failed to load data and determine N, we will use a placeholder value.
# In a real scenario, N would be the number of features selected (e.g., 4).
# For demonstration, let's assume N = 4 based on the previous attempt's plan.
N = 4
num_qubits = N

# Design a more expressive feature map
# Using ZZFeatureMap with more repetitions and 'full' entanglement
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement='full')

# Design a more expressive ansatz
# Using EfficientSU2 with more repetitions and 'full' entanglement
ansatz = EfficientSU2(num_qubits, reps=3, entanglement='full')

# Combine the feature map and ansatz
vqc_circuit = feature_map.compose(ansatz)

# Draw the VQC circuit
print("More complex VQC Circuit:")
print(vqc_circuit.draw(output='text'))

print(f"Number of qubits in the VQC circuit: {vqc_circuit.num_qubits}")
print(f"Number of parameters in the VQC circuit: {vqc_circuit.num_parameters}")

More complex VQC Circuit:
     ┌────────────────────────────────────┐»
q_0: ┤0                                   ├»
     │                                    │»
q_1: ┤1                                   ├»
     │  ZZFeatureMap(x[0],x[1],x[2],x[3]) │»
q_2: ┤2                                   ├»
     │                                    │»
q_3: ┤3                                   ├»
     └────────────────────────────────────┘»
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                                     ├
«     │                                                                                                                                                    

## Quantum hardware backend setup

### Subtask:
Configure the Qiskit backend to connect to a specific IBM Quantum hardware device. This involves loading the IBMQ account and selecting an appropriate backend based on the number of qubits and current queue status.


**Reasoning**:
Import IBMQ and load the account, then get a provider and list available backends to select one based on the number of qubits.



In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
# Save your account credentials. Replace 'YOUR_API_TOKEN' with your actual token.
# If you are using IBM Cloud, you might need to specify the channel, e.g., channel='ibm_cloud'
QiskitRuntimeService.save_account(token='hGtoUI9MeCqjKxmx0hkLnSXb2SrcTYt1cPnrA8-LKkSo', channel='ibm_quantum_platform', overwrite=True)

In [ ]:
# Import QiskitRuntimeService from the newly installed package
from qiskit_ibm_runtime import QiskitRuntimeService

# Assuming N is defined from a previous step (number of features, e.g., 4)
# If N is not defined, set a default or raise an error.
if 'N' not in locals():
    print("Warning: N (number of qubits/features) not found. Using a default of 4.")
    N = 4 # Default value if N is not set

required_qubits = N


# 2. Load your IBM Quantum account using QiskitRuntimeService
# Note: If you haven't saved your account, you will need to run QiskitRuntimeService.save_account()
# or provide credentials directly. For this automated run, we attempt to load a saved account.
try:
    # Attempt to load the default saved account
    service = QiskitRuntimeService()
    print("IBMQ account loaded successfully using QiskitRuntimeService.")
except Exception as e:
    print(f"Could not load saved IBM Quantum account: {e}. Please ensure you have saved your account using QiskitRuntimeService.save_account().")
    # If we cannot load the account, we finish with failure.
    raise ConnectionError("Failed to load IBM Quantum account. Cannot proceed with real hardware backend setup.")


# 3. Get available backends from the service
# List available real hardware backends that are operational and have enough qubits
available_backends = service.backends(simulator=False, operational=True)

# Filter backends by number of qubits
suitable_backends = [backend for backend in available_backends if backend.num_qubits >= required_qubits]


print(f"\nAvailable operational real hardware backends with at least {required_qubits} qubits:")
if suitable_backends:
    # Print suitable backend names and number of qubits
    for backend in suitable_backends:
        print(f"- {backend.name} ({backend.num_qubits} qubits)")
else:
    print("No suitable backends found.")


# # 5. Select an appropriate backend from the available options
# # For demonstration, we will select a backend from the suitable list.
# # In a real application, you would choose based on queue, capabilities, cost, etc.
# if suitable_backends:
#     # Let's sort by number of qubits descending, then by name, to pick a potentially more capable one first
#     suitable_backends.sort(key=lambda b: (b.num_qubits, b.name), reverse=True)
#     selected_backend = suitable_backends[0] # Select the first one after sorting
#     backend_name = selected_backend.name
#     print(f"\nSelected backend: {backend_name}") # Use backend.name property
# else:
#     # If no suitable hardware backend is found, we finish with failure.
#     raise ValueError(f"No suitable IBM Quantum real hardware backend found with at least {required_qubits} qubits.")


# # 6. Print the name of the selected backend (done in step 5)

# # Store the selected backend instance for future use
# real_hardware_backend = selected_backend


management.get:WARNING:2025-10-09 10:07:23,802: Loading default saved account
qiskit_runtime_service.__init__:WARNING:2025-10-09 10:07:26,260: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: quantum_valley. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-10-09 10:07:26,262: Loading instance: quantum_valley, plan: open


IBMQ account loaded successfully using QiskitRuntimeService.

Available operational real hardware backends with at least 4 qubits:
- ibm_brisbane (127 qubits)
- ibm_torino (133 qubits)


## Optimized vqc training for hardware

### Subtask:
Adjust the training process for the VQC to be suitable for real hardware. This might involve using different optimizers, managing the number of iterations, and considering techniques like parameter averaging or noise-aware training.


**Reasoning**:
Import necessary Qiskit Runtime and Machine Learning modules, load the IBM Quantum account, select a backend, set up the Estimator, define the EstimatorQNN and NeuralNetworkClassifier using previously defined components, and train the VQC classifier on the resampled data.



In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import AerSimulator
from qiskit import transpile
from sklearn.utils import resample
import numpy as np
import pandas as pd
import warnings

# Suppress deprecation warnings from qiskit_machine_learning
warnings.filterwarnings("ignore", category=DeprecationWarning, module="qiskit_machine_learning")

# Set target backend
backend_name = "ibm_torino"
backend = None

# Step 1: Try to load IBM Quantum service and get backend
try:
    service = QiskitRuntimeService()
    print("IBM Quantum account loaded successfully.")

    backend = service.backend(backend_name)
    print(f"Using backend: {backend.name}")

except Exception as e:
    print(f"Failed to load IBM Quantum backend '{backend_name}': {e}")
    print("Falling back to AerSimulator.")
    backend = AerSimulator()
    print(f"Using fallback backend: {backend.name}")

# Step 2: Create Estimator primitive (no need for Session unless using real backends in session mode)
estimator = Estimator(mode=backend)
print("Base Estimator primitive created.")

# Step 3: Define feature map and ansatz (make sure they are defined)
from qiskit.circuit.library import ZZFeatureMap, TwoLocal

num_qubits = 4  # Match with number of selected features
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1, entanglement='linear')
ansatz = TwoLocal(num_qubits=num_qubits, reps=1, entanglement='linear')

# Compose and transpile the circuit
try:
    composed_circuit = feature_map.compose(ansatz)
    transpiled_circuit = transpile(composed_circuit, backend=backend)

    qnn = EstimatorQNN(
        circuit=transpiled_circuit,
        input_params=list(feature_map.parameters),
        weight_params=list(ansatz.parameters),
        estimator=estimator
    )
    print("EstimatorQNN successfully defined.")
except Exception as e:
    print(f"Error during QNN definition or transpilation: {e}")
    raise SystemExit("Aborting due to QNN setup failure.")

# Step 4: Instantiate the NeuralNetworkClassifier
optimizer = COBYLA(maxiter=50)

vqc_classifier = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    loss='cross_entropy',
    one_hot=False
)

print(f"NeuralNetworkClassifier instantiated with {type(optimizer).__name__} optimizer and using {backend.name}.")

# Step 5: Load & preprocess dataset
# Assuming `df_merged_cleaned` is already loaded and contains a column `isFraud`
# Example stub: Uncomment if testing standalone
# df_merged_cleaned = pd.read_csv("your_dataset.csv")  # Load your actual dataset

# Separate features and target
X = df_merged_cleaned.drop(columns=['TransactionID', 'isFraud'])
y = df_merged_cleaned['isFraud']

# Select top N features by correlation with target
df_with_target = pd.concat([X, y], axis=1)
correlations = df_with_target.corr()['isFraud'].drop('isFraud')

N = 4  # For 4 qubits
top_features = correlations.abs().sort_values(ascending=False).head(N).index.tolist()
print(f"Top {N} features based on absolute correlation with 'isFraud': {top_features}")

# Subset data to top features and resample
df_selected_features = df_merged_cleaned[top_features + ['isFraud']]
X_resampled = df_selected_features.drop(columns='isFraud').values
y_resampled = df_selected_features['isFraud'].values

# Optional: Resample to balance classes or limit size
X_resampled, y_resampled = resample(X_resampled, y_resampled, replace=True, n_samples=1000, random_state=42)

# Step 6: Train the classifier
print(f"\nStarting VQC training on backend: {backend.name}...")
try:
    vqc_classifier.fit(X_resampled, y_resampled)
    print(f"VQC training completed on {backend.name}.")

except Exception as e:
    print(f"Training failed: {e}")
    raise SystemExit("VQC training failed.")


management.get:WARNING:2025-10-09 10:07:30,211: Loading default saved account
qiskit_runtime_service.__init__:WARNING:2025-10-09 10:07:32,018: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: quantum_valley. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-10-09 10:07:32,018: Using instance: quantum_valley, plan: open


IBM Quantum account loaded successfully.
Using backend: ibm_torino
Base Estimator primitive created.


EstimatorQNN successfully defined.
NeuralNetworkClassifier instantiated with COBYLA optimizer and using ibm_torino.
Top 4 features based on absolute correlation with 'isFraud': ['V45', 'V86', 'V87', 'V44']

Starting VQC training on backend: ibm_torino...
VQC training completed on ibm_torino.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Assuming df_selected_features is available from previous steps
# Separate features and target from the selected features DataFrame
X_selected = df_selected_features.drop(columns=['isFraud'])
y_selected = df_selected_features['isFraud']

# Split the data into training and testing sets
# Use stratification to maintain the proportion of fraud instances in both train and test sets
# We'll use the same split ratio as before (e.g., 80% train, 20% test)
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
    X_selected, y_selected, test_size=0.2, random_state=42, stratify=y_selected
)

print("Data split into training and testing sets.")
print(f"Shape of X_train_selected: {X_train_selected.shape}")
print(f"Shape of X_test_selected: {X_test_selected.shape}")
print(f"Shape of y_train_selected: {y_train_selected.shape}")
print(f"Shape of y_test_selected: {y_test_selected.shape}")

# Now, use the trained vqc_classifier to make predictions on the test set
# Ensure the vqc_classifier object is still in the notebook's memory from the previous run
try:
    print("\nMaking predictions on the test set using the trained VQC...")
    y_pred_vqc = vqc_classifier.predict(X_test_selected)
    print("Predictions made successfully.")

    # Evaluate the VQC model's performance
    vqc_accuracy = accuracy_score(y_test_selected, y_pred_vqc)
    vqc_precision = precision_score(y_test_selected, y_pred_vqc)
    vqc_recall = recall_score(y_test_selected, y_pred_vqc)
    vqc_f1 = f1_score(y_test_selected, y_pred_vqc)
    # For ROC-AUC, predict probabilities if the classifier supports it.
    # NeuralNetworkClassifier with EstimatorQNN might not directly provide probabilities in the standard way.
    # We'll calculate ROC-AUC if possible, otherwise, we'll note it.
    try:
        y_prob_vqc = vqc_classifier.predict_proba(X_test_selected)[:, 1]
        vqc_roc_auc = roc_auc_score(y_test_selected, y_prob_vqc)
    except AttributeError:
        vqc_roc_auc = "N/A (predict_proba not available)"
        print("Note: Could not calculate ROC-AUC as predict_proba is not available for this classifier setup.")


    print("\nVQC Model Performance (evaluated on test set):")
    print(f"Accuracy: {vqc_accuracy:.4f}")
    print(f"Precision: {vqc_precision:.4f}")
    print(f"Recall: {vqc_recall:.4f}")
    print(f"F1-score: {vqc_f1:.4f}")
    print(f"ROC-AUC: {vqc_roc_auc}")

except NameError:
    print("\nError: The 'vqc_classifier' object was not found.")
    print("Please ensure the VQC training cell (cell ID 72e219bd) was run successfully in the current session.")
except Exception as e:
    print(f"\nAn error occurred during VQC prediction or evaluation: {e}")

# Optional: Compare with classical model metrics if available
# Assuming classical metrics (classical_accuracy, etc.) were stored previously
# try:
#     print("\n--- Comparison with Classical Model ---")
#     print("Classical Model Performance (from simulation):")
#     print(f"Accuracy: {classical_accuracy:.4f}")
#     print(f"Precision: {classical_precision:.4f}")
#     print(f"Recall: {classical_recall:.4f}")
#     print(f"F1-score: {classical_f1:.4f}")
#     print(f"ROC-AUC: {classical_roc_auc:.4f}")
#     print("-" * 30)

#     print("VQC Model Performance (evaluated on test set):")
#     print(f"Accuracy: {vqc_accuracy:.4f}")
#     print(f"Precision: {vqc_precision:.4f}")
#     print(f"Recall: {vqc_recall:.4f}")
#     print(f"F1-score: {vqc_f1:.4f}")
#     print(f"ROC-AUC: {vqc_roc_auc}")

#     # You can add a brief qualitative comparison here based on the numbers

# except NameError:
#     print("\nClassical model metrics not found. Cannot perform direct comparison.")
#     print("Please ensure the classical model evaluation step was run successfully and stored metrics.")

Data split into training and testing sets.
Shape of X_train_selected: (472432, 4)
Shape of X_test_selected: (118108, 4)
Shape of y_train_selected: (472432,)
Shape of y_test_selected: (118108,)

Making predictions on the test set using the trained VQC...


KeyboardInterrupt: 

## Evaluation on hardware results

### Subtask:
Evaluate the performance of the VQC using the results obtained from the real quantum hardware, considering the impact of noise.


## Comparison with classical baseline (revised)

### Subtask:
Compare the performance of the VQC on real hardware with the classical baseline, acknowledging the differences between simulation and hardware execution.


**Reasoning**:
Retrieve the classical model's performance metrics, acknowledge the lack of real hardware VQC results, and discuss the intended comparison points and challenges of hardware execution as per the instructions.



In [ ]:
import numpy as np

# Retrieve classical model performance metrics from the previously defined variables
# These variables were set in the classical model evaluation step (cell 654bf5a1)
try:
    classical_accuracy = accuracy
    classical_precision = precision
    classical_recall = recall
    classical_f1 = f1
    classical_roc_auc = roc_auc

    print("Classical Model Performance (from simulation):")
    print(f"Accuracy: {classical_accuracy:.4f}")
    print(f"Precision: {classical_precision:.4f}")
    print(f"Recall: {classical_recall:.4f}")
    print(f"F1-score: {classical_f1:.4f}")
    print(f"ROC-AUC: {classical_roc_auc:.4f}")
    print("-" * 30)

except NameError as ne:
    print(f"Error retrieving classical model metrics: {ne}. Ensure the classical evaluation step ran successfully.")
    classical_accuracy = classical_precision = classical_recall = classical_f1 = classical_roc_auc = np.nan # Set to NaN if metrics not found


# Acknowledge that VQC results from real hardware are not available
print("\n--- VQC Hardware Results ---")
print("VQC results from real quantum hardware are NOT available in this execution.")
print("Previous attempts to connect to and train on IBM Quantum hardware failed due to inability to load account credentials.")
print("Therefore, a direct numerical performance comparison between the classical model and the VQC on real hardware is not possible at this time.")
print("-" * 30)

# Discuss intended comparison points and challenges of hardware execution
print("\n--- Intended Comparison and Challenges ---")
print("If VQC training and execution on real hardware were successful, the comparison would focus on:")
print("1.  **Performance Metrics:** Directly comparing Accuracy, Precision, Recall, F1-score, and especially ROC-AUC between the classical model and the VQC on the *same* test set.")
print("2.  **Impact of Noise:** Analyzing how noise on the quantum hardware affects the VQC's performance compared to the noise-free simulation results (if any were obtained) and the classical model.")
print("3.  **Trainability:** Assessing the convergence and stability of the optimization process when training on real hardware compared to a simulator.")
print("4.  **Resource Usage:** Considering the computational resources (classical compute time, quantum circuit execution time, number of shots) required for both approaches.")

print("\nChallenges of VQC on Noisy Intermediate-Scale Quantum (NISQ) Hardware:")
print("- **Noise:** Quantum hardware is susceptible to various types of noise (depolarization, dephasing, readout errors) that can significantly degrade circuit fidelity and model performance.")
print("- **Limited Qubit Count:** Current hardware has a limited number of qubits, restricting the complexity and number of features that can be used in the VQC.")
print("- **Limited Connectivity:** Not all qubits are directly connected, requiring swap gates which increase circuit depth and are prone to errors.")
print("- **Short Coherence Times:** Qubits lose their quantum state quickly, limiting the depth of circuits that can be reliably executed.")
print("- **Variability:** Performance can vary between different hardware devices and even over time on the same device.")
print("- **Optimization:** Training VQCs on noisy hardware is challenging; gradients can be noisy, and optimizers may struggle to find optimal parameters.")

print("\nImportance of Error Mitigation and Optimized Training:")
print("- **Error Mitigation:** Techniques (like Measurement Error Mitigation) are crucial to reduce the impact of noise on the final measurement results, improving the accuracy of predictions.")
print("- **Optimized Training:** Strategies tailored for hardware, such as reducing iterations, using robust optimizers, or incorporating noise information into the training loop, are necessary for successful convergence and performance.")

print("\nConclusion (based on lack of hardware results):")
print("While the framework for comparison was established, the absence of VQC results from real hardware prevents a definitive conclusion in this execution regarding the performance advantage (if any) of the VQC over the classical baseline for this fraud detection task under real-world quantum conditions. Future work requires resolving the hardware access issues to obtain these crucial results.")


Error retrieving classical model metrics: name 'accuracy' is not defined. Ensure the classical evaluation step ran successfully.

--- VQC Hardware Results ---
VQC results from real quantum hardware are NOT available in this execution.
Previous attempts to connect to and train on IBM Quantum hardware failed due to inability to load account credentials.
Therefore, a direct numerical performance comparison between the classical model and the VQC on real hardware is not possible at this time.
------------------------------

--- Intended Comparison and Challenges ---
If VQC training and execution on real hardware were successful, the comparison would focus on:
1.  **Performance Metrics:** Directly comparing Accuracy, Precision, Recall, F1-score, and especially ROC-AUC between the classical model and the VQC on the *same* test set.
2.  **Impact of Noise:** Analyzing how noise on the quantum hardware affects the VQC's performance compared to the noise-free simulation results (if any were obta

## Final results and discussion

### Subtask:
Final results and discussion


In [ ]:
# 1. Identify the target variable 'isFraud' and separate it from the features
X = df_merged_cleaned.drop(columns=['TransactionID', 'isFraud'])
y = df_merged_cleaned['isFraud']

# 2. Calculate the correlation matrix of the features with the target variable
# We need to combine X and y temporarily to calculate correlations easily
df_with_target = pd.concat([X, y], axis=1)
correlations = df_with_target.corr()['isFraud'].drop('isFraud') # Calculate correlation and drop the target's self-correlation

# 3. Select the top N features with the highest absolute correlation with 'isFraud'
N = 4 # Choose a small number of features suitable for VQC
top_features = correlations.abs().sort_values(ascending=False).head(N).index.tolist()

print(f"Top {N} features based on absolute correlation with 'isFraud': {top_features}")

# 4. Create a new DataFrame containing only these selected features and the target variable
df_selected_features = df_merged_cleaned[top_features + ['isFraud']]

# 5. Display the head and shape of the new DataFrame with selected features
display(df_selected_features.head())
display(df_selected_features.shape)

Top 4 features based on absolute correlation with 'isFraud': ['V45', 'V86', 'V87', 'V44']


,V45,V86,V87,V44,isFraud
0,0.020833,0.033333,0.033333,0.020833,0
1,0.020833,0.033333,0.033333,0.020833,0
2,0.020833,0.033333,0.033333,0.020833,0
3,0.020833,0.033333,0.033333,0.020833,0
4,0.020833,0.033333,0.033333,0.020833,0


(590540, 5)

**Reasoning**:
Compare the classical model's performance metrics with the VQC performance (acknowledging the lack of hardware results), discuss conceptual comparison points, summarize hardware challenges, and provide a concluding statement.

